In [4]:
import torch
from torch.autograd import Variable


In [5]:
x = Variable(torch.ones(2,2).cuda(), requires_grad=True)

y = x+2
# print(y.grad_fn)

z = y*y*3
out = z.mean()

print(z,out)

Variable containing:
 27  27
 27  27
[torch.cuda.FloatTensor of size 2x2 (GPU 0)]
 Variable containing:
 27
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [6]:
out.backward()
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.cuda.FloatTensor of size 2x2 (GPU 0)]



In [12]:
x = torch.randn(3).cuda()
x = Variable(x, requires_grad=True)

y =x*2
n = 2
while y.data.norm() < 1000:
    n*=2
    y=y*2
    
print(n)  
print(y)

1024
Variable containing:
 -764.2685
 -459.6710
 1019.8314
[torch.cuda.FloatTensor of size 3 (GPU 0)]



In [14]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]



## Neural Network

In [19]:
%matplotlib notebook
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [20]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', 
                        train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                         shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', 
                        train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                         shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
          'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [21]:
def imshow(img):
    img = img /2 + 0.5
    npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1,2,0)))
    
dataiter = iter(trainloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))

print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

  car plane  frog  ship


In [7]:
#defining network

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)

In [17]:
net = Net()
net = net
print(net)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

def train():
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = net(inputs) #forward
        loss = criterion(outputs, labels) #set up loss
        loss.backward() #backward
        optimizer.step() #optimize gradients
        
        running_loss += loss.data[0]
        if(i%2000 == 1999):
            print('[%d, %5d] loss: %.3f' % (epoch+1,i+1,
                                            running_loss/2000))
            running_loss = 0.0

def test():
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Accuracy of the network on the 10000 test images: %d %%' % (100*correct/total))

Net(
  (conv1): Conv2d (3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


In [12]:
for epoch in range(2):
    train()

[1,  2000] loss: 1.198
[1,  4000] loss: 1.172
[1,  6000] loss: 1.176
[1,  8000] loss: 1.159
[1, 10000] loss: 1.151
[1, 12000] loss: 1.170
[2,  2000] loss: 1.093
[2,  4000] loss: 1.093
[2,  6000] loss: 1.097
[2,  8000] loss: 1.087
[2, 10000] loss: 1.082
[2, 12000] loss: 1.089


In [18]:
test()

Accuracy of the network on the 10000 test images: 10 %


In [81]:
def test():
    correct = 0
    total = 0
    for data in testloader:
        images, labels = data
        ouputs = net(Variable(images))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Accuracy of the network on the 10000 test images: %d %%' % (100*correct/total))